# Data generation for ANN recognition

This notebook generates unmasked and masked data for the ANN recognition experiments described in "Section 6: Validation Experiment: Human visual selectivity Boosts ANN Recognition." To begin, unzip the human map data into the folder `human-maps/`.

In [ ]:
import numpy as np
from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy import ndimage
import matplotlib.pyplot as plt
import fnmatch
import os
import itertools
import cv2

In [ ]:
# data directory path
data_dir = dirname('human-maps/')
# files in the directory
dir_fls = os.listdir(data_dir)

################################
######### IMAGE FILES ##########
################################

# get list of images
image_fls = [fl for fl in dir_fls if fnmatch.fnmatch(fl, '*image.mat')]  
image_fls = sorted(image_fls)

################################
####### HUMAN DATA FILES #######
################################

PC_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*humanPC.mat')])
ptch_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*patchRate.mat')])
disc_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*discr.mat')])
KDE_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*chainKDE.mat')])
feye_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*freeFIX.mat')])
seye_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*salFIX.mat')])
oeye_fls = sorted([fl for fl in dir_fls if fnmatch.fnmatch(fl, '*objFIX.mat')])

Helper functions for loading, saving, and mask/image processing.

In [ ]:
which_files = {
    'ptch': ptch_fls,
    'disc': disc_fls,
    'KDE':  KDE_fls,
    'feye': feye_fls,
    'seye': seye_fls,
    'oeye': oeye_fls,
    'PC': PC_fls,
}


def get_mask(mask_fl, exponent, threshold, rotation, mp_shape):
    """Load the mask and apply transformations."""

    # Path to specific (patch) map.
    mat_fname = pjoin(data_dir, mask_fl)
    mat_contents = sio.loadmat(mat_fname)
    mp = mat_contents['map']
    
    # Resize.
    mp = cv2.resize(mp, mp_shape, interpolation=cv2.INTER_AREA)
    
    # Truncate.
    if threshold is None:
        # Truncate 50% of pixels.
        half = np.prod(mp.shape) // 2
        threshold = np.partition(mp, np.prod(mp.shape)//2, axis=None)[half]
    mp[mp < threshold] = threshold
    mp -= threshold

    # Normalize.
    mp /= np.max(mp)

    # Exponentiate.
    mp **= exponent
    
    # Rotate.
    mp = np.rot90(mp, rotation // 90)
    
    return mp
 
    
def get_image(image_fl, im_shape):
    """Load the image, resize, and tranform to float."""
    
    # Read in image.
    im_fname = pjoin(data_dir, image_fl)
    im_contents = sio.loadmat(im_fname)
    im = im_contents['im']

    # Resize.
    im = cv2.resize(im, im_shape, interpolation=cv2.INTER_AREA)

    # Color conversion.
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(float)/255 
    
    return im
        

def create_test_data(mask_type, null_type, exponent, threshold, rotation, im_shape=[250, 250]):
    
    # One folder per hparam setting.
    set_name = 'mask={}_null={}_exp={}_threshold={}_rotation={}'.format(
        mask_type, null_type, exponent, threshold, rotation)
    dir_name = 'NN_recognition_data'
    set_fn = pjoin(dir_name, set_name)
         
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    if not os.path.exists(set_fn):
        os.mkdir(set_fn)
    
    # Combine the image with each mask (correct and incorrect).
    id_mask_fls = which_files[mask_type]
    combos = list(itertools.product(image_fls, id_mask_fls))

    for image_fl, mask_fl in combos:
        mp = get_mask(mask_fl, exponent, threshold, rotation, mp_shape=im_shape)
        im = get_image(image_fl, im_shape=im_shape)

        if null_type == 'black':
            null_value = np.array([0.0, 0.0, 0.0])
        elif null_type == 'average':
            null_value = im.mean(axis=0).mean(axis=0)
        else:
            raise ValueError("Unsupported null value type.")

        mask_id = mask_fl.split('_')[0][3:]
        im_id = image_fl.split('_')[0][3:]

        # Interpolate to null value.
        im = (
            np.einsum('ij,k->ijk', 1 - mp, null_value) +
            np.einsum('ij,ijk->ijk', mp, im - null_value[np.newaxis, np.newaxis, :])
        )

        # Write image to File
        fn = 'img{}_mask{}.png'.format(im_id, mask_id)        
        file_path = pjoin(set_fn, fn)
        cv2.imwrite(file_path, im * 255)
        
    # Also create an appropriately-resized unmasked image.
    # This assumes all masks of a certain type are the same size.
    for image_fl in image_fls:
        im = get_image(image_fl, im_shape=mp.shape)
        imid = image_fl.split('_')[0][3:]
        fn = 'img{}_nomask.png'.format(imid)   
        file_path = pjoin(set_fn, fn)
        cv2.imwrite(file_path, im * 255)

Create the unmasked and masked data. 

In [ ]:
null_types = ['black']#, 'average']
mask_types = which_files.keys()
exponents = [1.0]#, 1.5, 2.0, 4.0, 8.0, 16.0]
thresholds = [None] #[0.0625, 0.125, 0.25, 0.5]
rotations = [0, 90, 180, 270]

for (mask_type, null_type, exponent, threshold, rotation) in itertools.product(
    mask_types, null_types, exponents, thresholds, rotations):
    create_test_data(mask_type=mask_type, 
                     null_type=null_type,
                     exponent=exponent,
                     threshold=threshold,
                     rotation=rotation)